<a href="https://colab.research.google.com/github/lcbjrrr/genai/blob/main/06_RAG_Guard_Eval_GCPpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG GuardRails and Evaluation


## VectorDB Re-creation

In [ ]:


from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
import os
from google.colab import userdata

API_KEY='your key'
os.environ["GOOGLE_API_KEY"] = API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e=embeddings.embed_documents("Python is the best to do LLM")
print(e[-1])
len(e[-1])

import os
files = []
for file in os.listdir('./'):
    if file.endswith('.pdf'):
        files.append(file)

files[0]

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
def load_pdf(filename):
    loader = UnstructuredPDFLoader(filename,mode="elements")
    return loader.load()

def chunk(filename,chunks_all):
  print('Processing',filename)
  elements = load_pdf('./'+filename)
  docs=[]
  for e in elements:
    meta = e.metadata.copy()
    docs.append(Document(page_content=e.page_content, metadata=meta))
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
  chunks = text_splitter.split_documents(docs)
  #print(chunks[10])
  #print(len(docs),len(chunks))
  #docs[8].page_content
  print(chunks)
  chunks_all.extend(chunks)
  print('------ Processed',len(docs),len(chunks))

all_chunks = []
for file in files:
  chunk(file,all_chunks)

from langchain_community.vectorstores.utils import filter_complex_metadata
print(len(all_chunks))
all_chunks=filter_complex_metadata(all_chunks)
len(all_chunks)

print(all_chunks[10].page_content)


In [ ]:
from langchain_community.vectorstores.utils import filter_complex_metadata
print(len(all_chunks))
all_chunks=filter_complex_metadata(all_chunks)
len(all_chunks)

print(all_chunks[10].page_content)

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
#embed_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embed_model = GoogleGenerativeAIEmbeddings(model="embedding-001")
i=0
vectorstore = Chroma(embedding_function=embed_model,persist_directory='./vectordb')
for chunk in all_chunks:
  print(i,'=>',chunk.metadata.get('filename'),chunk.metadata.get('element_id'))
  try:
      vectorstore.add_documents([chunk])
  except Exception as e:
      print('#### Error ',e)
  i+=1

In [ ]:
import langchain
print(langchain.__version__)

1.0.5


## Guardrails

As salvaguardas são um conjunto crucial de medidas de segurança projetadas para garantir que o sistema opere de forma segura, ética e dentro de limites definidos. Agindo como as barreiras físicas em uma rodovia, esses mecanismos não afetam a inteligência central do modelo, mas monitoram e filtram ativamente tanto a entrada do usuário quanto a saída do modelo para evitar comportamentos indesejáveis. Seu principal objetivo é mitigar riscos como a geração de conteúdo tóxico ou tendencioso, a exposição de informações sensíveis (PII), a produção de imprecisões factuais (alucinações) ou a vulnerabilidade a ataques de injeção de código e jailbreak que tentam burlar as configurações de segurança. Ao implementar camadas de verificações, classificadores de conteúdo e aplicação de políticas, as salvaguardas ajudam os desenvolvedores a alinhar sistemas de IA poderosos com padrões organizacionais, conformidade legal e valores humanos, garantindo uma implantação responsável em aplicações do mundo real.

In [ ]:

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict, Annotated


results = {
    'question': [],
    'answer': [],
    'contexts': [],
    #'ground_truth': []
}

def query_vector_db(db,q,k):
  results = db.similarity_search_with_score(q, k=k)
  if results:
    return results
  else:
    print("No documents found matching the filter and query.")
    return None

LLM_MODEL_ID="gemini-2.5-flash"
NMSG = 5

class ChatState(TypedDict):
    messages: Annotated[list[HumanMessage | AIMessage | SystemMessage], add_messages]

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Define safety settings for the Gemini model
# These settings control the probability of blocking content for different harm categories.
# BLOCK_NONE: Blocks no content.
# BLOCK_ONLY_HIGH: Blocks only content that is deemed to have a high probability of being harmful.
# BLOCK_MEDIUM_AND_ABOVE: Blocks content that is deemed to have a medium or high probability of being harmful.
# BLOCK_LOW_AND_ABOVE: Blocks content that is deemed to have a low, medium, or high probability of being harmful.
guardrails = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    # You can also set a default threshold for all categories not explicitly listed
    # "default_threshold": HarmBlockThreshold.BLOCK_NONE, # Example: no blocking by default
}

llm = ChatGoogleGenerativeAI(model=LLM_MODEL_ID, temperature=0.0,safety_settings=guardrails)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

def call_model_node(state: ChatState):
    last_messages = state["messages"][(NMSG-1)*-2:]
    hist = ''
    for h in last_messages:
      hist +=  h.content + '\n'

    last_user_msg = [m for m in last_messages if isinstance(m, HumanMessage)][-1].content
    #relevant_docs = retriever.get_relevant_documents(last_user_msg)
    #doc_messages = [SystemMessage(content=f"Relevant doc: {doc.page_content}") for doc in relevant_docs]
    doc_messages=''
    contexts=[]
    resp = query_vector_db(vectorstore,question,3)
    for r in resp:
      doc_messages += '- '+r[0].page_content+'\n'
      contexts.append(r[0].page_content)
    p = '''
    Considering these documents as a starting point:
    {docs}
    In the context of the history of this conversation:
    {history}
    Answer this question:
    {question}
    '''
    prompt = p.format(docs=doc_messages, history=hist,question=last_user_msg)
    print('---------------------')
    print(prompt)
    print('---------------------')
    response = llm.invoke(prompt)

    results['question'].append(question)
    results['answer'].append(response.content)
    results['contexts'].append(contexts)
    #results['ground_truth'].append(dados_teste['ground_truth'][i])

    return {"messages": [response]}

builder = StateGraph(ChatState)
builder.add_node("model", call_model_node)
builder.add_edge(START, "model")
builder.add_edge("model", END)
graph = builder.compile(checkpointer=InMemorySaver())

def chat(session_id: str, user_input: str):
    human_msg = HumanMessage(content=user_input)
    result = graph.invoke({"messages": [human_msg]},
        {"configurable": {"thread_id": session_id}})
    ai_msg = result["messages"][-1]
    return ai_msg.content, result["messages"]

question = input("Enter your question/prompt: ")
while(question!='exit'):
  response, history = chat("default", question)
  print(response)
  question = input("Enter your question/prompt: ")

print("========== Full History: ==========")
for message in history:
    print('-----------------------------------')
    print(message.content)

## Model Evaluation (with RAGAS)

RAGAS (Retrieval-Augmented Generation Assessment) é uma estrutura de avaliação especificamente projetada para medir a qualidade de sistemas de Geração Aumentada por Recuperação (RAG) sem depender de extensos dados de referência anotados por humanos. Duas de suas principais métricas de geração são Fidelidade e Relevância da Resposta. A Fidelidade mede a consistência factual da resposta gerada em relação ao contexto específico recuperado pelo sistema RAG. Uma pontuação alta aqui indica uma baixa taxa de alucinação, o que significa que a resposta se atém estritamente aos fatos fornecidos nos documentos de origem. A Relevância da Resposta, por outro lado, avalia o quão pertinente e completa a resposta gerada é para a pergunta original do usuário. Ela penaliza respostas incompletas, que contêm informações redundantes ou que se desviam do tópico da consulta, garantindo que o modelo RAG atenda à intenção do usuário de forma direta e eficiente.

In [ ]:
print("========== Full History: ==========")
for message in history:
    print('-----------------------------------')
    print(message.content)

In [ ]:
!pip install ragas datasets

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

eval_model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.0 )

ragas_ds = Dataset.from_dict(results)
ragas_metrics = [faithfulness,answer_relevancy]

print("🔍 Starting RAGAS...")
ragas_results = evaluate(ragas_ds,
    metrics=ragas_metrics,
    llm=eval_model,
    embeddings=embeddings)

print("\n✅ RAGAS Done!")
print(f"faithfulness: {ragas_results['faithfulness'][0]:.4f}")
print(f"answer_relevancy: {ragas_results['answer_relevancy'][0]:.4f}")
#print(f"context_precision: {ragas_results['context_precision'][0]:.4f}")
#print(f"context_recall: {ragas_results['context_recall'][0]:.4f}")



---



# Atividade: Guardrails a Avaliacao


Atividade: Guardrails a Avaliacao

Como engenheiro de IA responsável pelo assistente virtual corporativo baseado em RAG, você notou que o modelo ocasionalmente gera respostas alucinadas ou viola políticas de segurança interna; sua tarefa é implementar uma camada de guardrails programáticos para filtrar entradas e saídas inadequadas
Em seguida, estruturar um pipeline de testes automatizados com o framework RAGAS para quantificar a eficácia dessa implementação, comparando métricas de fidelidade (faithfulness) e relevância da resposta antes e depois da aplicação das proteções
